In [1]:
minmaxInterval = 7000

## open port

In [2]:
!echo 'qwer1234'| sudo -S chmod a+rw /dev/ttyUSB0

[sudo] password for terry-q: 

## BASIC USAGE OF XL320 CLASS

In [3]:
import os, time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from math import pi as PI
DEG90 = PI/2
PROPORTION_MATRIX = np.array(
    [
        [1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.1,1,0.4,0.4, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        [0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0,  1,0.1,0.4,0.4, 0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.1,1,0.4,0.4,  0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,1,0.1, 0,0,0,0, 0,0,0,0],
        [0,0,0,0,  0.4,0.4,0.1,1, 0,0,0,0, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1, 0,0,0,0],
        [0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1, 0,0,0,0],
        
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 1,0.1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.1,1,0.4,0.4],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.4,0.4,1,0.1],
        [0,0,0,0, 0,0,0,0, 0,0,0,0, 0.4,0.4,0.1,1],
        ])


In [4]:
import sys
sys.path.append(".")
""" FOR DYNAMIXEL """
from model.class_xc330 import xc330
""" FOR MODERN DRIVER """
import roslib; roslib.load_manifest('ur_driver')
import rospy
import actionlib
from control_msgs.msg import *
from trajectory_msgs.msg import *
from sensor_msgs.msg import JointState


class FourFingers(object):
    def __init__(self,):
        rospy.init_node("DEMO")
        self.JOINT_NAMES = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
                            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
        self.client      = None
        
        self.xc          = xc330('SOFTROBOT', _USB_NUM=0)
        
    def init_xc(self,initpos):
        self.xc.connect()
        self.xc.set_operatingmode([4])

        self.xc.set_torque([0])
        time.sleep(2)
        self.xc.set_torque([1])
        print ("TORQUE ON")

        currpos = self.xc.get_currpos()
        minmaxInterval = 5000
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)

        if (initpos==None).any(): initpos = currpos; print("No initpos Found")
        self.initpos = initpos
        self.xc.set_goalposcluster(initpos,1)

        print ("INITIALIZE POSITION")
        currpos = self.xc.get_currpos()
        print (currpos)

        minmaxInterval = minmaxInterval
        self.xc.set_minmaxpos(currpos-np.ones_like(currpos)*minmaxInterval,currpos+np.ones_like(currpos)*minmaxInterval)
    
    def move_xc(self,pos):
        self.xc.set_goalposcluster(pos,1)
        
    def pmove_xc(self,pos):
        pos = self.initpos + PROPORTION_MATRIX @ np.array(pos) * np.array([1,-1,-1,1]*4)
        self.xc.set_goalposcluster(pos,1)
        
    def move_arm(self, joints):
        try: 
            q = joints
            g = FollowJointTrajectoryGoal()
            g.trajectory = JointTrajectory()
            g.trajectory.joint_names = self.JOINT_NAMES
            joint_states = rospy.wait_for_message("joint_states", JointState)
            joints_pos   = joint_states.position
            g.trajectory.points = [
                JointTrajectoryPoint(positions=joints_pos, velocities=[0]*6, time_from_start=rospy.Duration(0.0)),
                JointTrajectoryPoint(positions=q, velocities=[0]*6, time_from_start=rospy.Duration(3))]  
            self.client.send_goal(g)
            self.client.wait_for_result()
        except KeyboardInterrupt:
            self.client.cancel_goal()
            raise
        except:
            raise  
         
    def execute_arm(self, joints):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm(joints)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      


['/home/terry-q/Rilab/sOftrobot/SORO-Dynamixel-python', '', '/home/terry-q/catkin_ws/devel/lib/python3/dist-packages', '/opt/ros/noetic/lib/python3/dist-packages', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '/home/terry-q/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '.']


## INSTANTIATE Robot 

In [5]:
robot = FourFingers()

[SOFTROBOT] INSTANTIATED AT [b'/dev/ttyUSB0']


In [6]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
robot.execute_arm(init_joint)

Waiting for server...
Connected to server
Finish plan


# Calibrate initpos
반드시 Cap을 벗기고 진행할것

In [ ]:
# Upright_joint = np.array([0, -90+45, 90-45, -90, 90, 90])/90*DEG90
# robot.execute_arm(Upright_joint)

In [ ]:
# robot.init_xc(initpos=np.array([None]))
# initpos = robot.xc.get_currpos()

In [ ]:
# relaxed_pos = initpos

In [ ]:
# initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
#        1990,  2605, -703, 2454, 1732])
# # initpos[-4:] = relaxed_pos[-4:]
# robot.init_xc(initpos=initpos)

In [ ]:
# relaxed_pos

In [ ]:
# # 1번은 플러스, 2번 마이너스, 3번 마이너스, 4번 플러스
# pos = initpos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,0, 100,200,200,200]) * np.array([1,-1,-1,1]*4)
# # robot.move_xc(pos)

# pos = pos + PROPORTION_MATRIX @ np.array([0,0,0,0, 0,0,0,0, 0,0,0,0, 600,600,600,600]) * np.array([1,-1,-1,1]*4)

# robot.move_xc(pos)

# robot.xc.get_currpos()

In [ ]:
# initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
#        1990, 4110,  970, 3106, 3197])
# robot.init_xc(initpos)
# robot.xc.get_currpos()

# pos = initpos + np.array([300]*16) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)

# pos = initpos + PROPORTION_MATRIX @ np.array([1000,0,0,0, 0,0,1000,0, 0,1000,0,0, 0,0,0,1000]) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)


# pos = initpos + PROPORTION_MATRIX @ np.array([0,1000,0,0, 0,0,0,1000, 1000,0,0,0, 0,0,1000,0]) * np.array([1,-1,-1,1]*4)
# robot.move_xc(pos)
# time.sleep(2)

# robot.move_xc(initpos)

# Twist Open

In [7]:
robot.init_xc(np.array([None]))
initpos = robot.xc.get_currpos()
pos = initpos + np.array([1,1,1,1, 1,1,1,1, 1,1,1,1, 1,1,1,1])*1000 * np.array([1,-1,-1,1]*4)
robot.move_xc(pos)

initpos = np.array([2247, -546,  344, 2629, 1000, 2833, 1801, 4351, 4761, 2223, 1299,
       1990,  2705, -803, 2354, 1832])
robot.init_xc(initpos)

robot.xc.get_currpos()

[SOFTROBOT] CONNECT
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [16]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]]
TORQUE ON
No initpos Found
INITIALIZE POSITION
[1872  150  818 2040  574 3438 2144 3701 4350 3154 2029 1159 1867   54
 2864  908]
[SOFTROBOT] CONNECT
[PortHandler setup] The port number 0 has same device name... reinitialize port number 0!!
 SUCCEEDED TO OPEN THE PORT!
 SUCCEEDED TO CHANGE THE BAUDRATE!
COMMUNICATION SUCCESS
[SOFTROBOT] DETECT ID
[SOFTROBOT] [16]MOTORS DETECTED 
[SOFTROBOT] ID_LIST:[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]]
TORQUE ON
INITIALIZE POSITION
[2229 -515  367 2596  982 2861 1820 4319 4740 2264 1334 1954 2667 -762
 2390 1784]


array([2239, -532,  357, 2612,  991, 2848, 1812, 4332, 4750, 2242, 1316,
       1973, 2687, -784, 2374, 1808])

# Close Up

In [15]:
Target_up_joint = [0.001,-DEG90,DEG90,-DEG90,-DEG90,DEG90]

Target_down_joint = np.array([0.02,-86.79,106.58,-109.79, -90.04, 90.11])/90*DEG90

Target_down_joint2 = np.array([0.02,-87.48,104.85,-107.46, -90.03, 90.09])/90*DEG90
Target_down_joint = (Target_down_joint2+Target_down_joint)/2
assert (Target_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint2 * np.array([1,-1,1,-1,-1,1])>0).all()

robot.execute_arm(Target_up_joint)
robot.pmove_xc(np.array([0,0,0,0]*4))



Waiting for server...
Connected to server
Finish plan


In [16]:
coeff = 1.4
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

In [17]:
# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

robot.execute_arm(Target_down_joint)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# 들고
act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# 들고
act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

# Ungrasp
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# 접근 동작
act = rollout_actuation[1].tolist()
robot.pmove_xc(np.array(act*4))

# 잡는 동작
act = rollout_actuation[2].tolist()
robot.pmove_xc(np.array(act*4))

# 절반 돌리고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))

# 나머지 돌리고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))



Waiting for server...
Connected to server
Finish plan


In [18]:
robot.execute_arm(Target_up_joint)
time.sleep(4)
robot.execute_arm(Target_down_joint2)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [19]:
### Capping ###

# 놓고
act = rollout_actuation[3].tolist()
robot.pmove_xc(np.array(act*4))
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([0,2000,2000,0]*4))

# 잡는 동작
robot.pmove_xc(np.array([0,1800,200,800]*4))

# 절반 돌리고
robot.pmove_xc(np.array([500,500,200,1200]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([700,1000,200,2000]*4))

# 들고
robot.pmove_xc(np.array([3000,1000,0,2000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))


In [20]:
robot.execute_arm(Target_up_joint)

Waiting for server...
Connected to server
Finish plan


In [114]:
# # Ungrasp
# act = rollout_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)

# # 접근 동작
# act = rollout_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 잡는 동작
# act = rollout_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = rollout_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = rollout_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

In [91]:
robot.pmove_xc(np.array([1000,0,1000,0]*4))

In [87]:
# # 잡는 동작
# robot.pmove_xc(np.array([0,1800,200,800]*4))

# # 절반 돌리고
# robot.pmove_xc(np.array([800,800,500,1500]*4))

# # 나머지 돌리고
# robot.pmove_xc(np.array([500,800,0,1800]*4))

In [53]:
coeff = 0.7
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

cap_actuation = np.zeros_like(rollout_actuation)

cap_actuation[:,0] = rollout_actuation[:,2]
cap_actuation[:,1] = rollout_actuation[:,3]
cap_actuation[:,2] = rollout_actuation[:,0]
cap_actuation[:,3] = rollout_actuation[:,1]


In [57]:
# # robot.execute_arm(Target_down_joint)

# # Ungrasp
# act = cap_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)

# # robot.execute_arm(Target_down2_joint)

# # 접근 동작
# act = cap_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 잡는 동작
# act = cap_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = cap_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = cap_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

# # 들고
# act = cap_actuation[5].tolist()
# robot.pmove_xc(np.array(act*4))

# # Ungrasp
# act = cap_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)


KeyboardInterrupt: 

In [46]:
# 놓고
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

In [47]:
coeff = 1.5
rollout_actuation = np.load("actuations/Custom_grasp.npy") * coeff

# 놓고
act = rollout_actuation[0].tolist()
robot.pmove_xc(np.array(act*4)/coeff)

# robot.execute_arm(Target_down_joint)

# 들고
act = rollout_actuation[4].tolist()
robot.pmove_xc(np.array(act*4))

# # 원래대로 오고
# act = rollout_actuation[4].tolist()
# robot.pmove_xc(np.array(act*4))

# # 절반 돌리고
# act = rollout_actuation[3].tolist()
# robot.pmove_xc(np.array(act*4))

# # 나머지 돌리고
# act = rollout_actuation[2].tolist()
# robot.pmove_xc(np.array(act*4))

# # 들고
# act = rollout_actuation[1].tolist()
# robot.pmove_xc(np.array(act*4))

# # 놓고
# act = rollout_actuation[0].tolist()
# robot.pmove_xc(np.array(act*4)/coeff)


In [44]:
rollout_actuation = np.load("actuations/Custom_grasp.npy")
rollout_actuation = np.flip(rollout_actuation,axis=0)
rollout_actuation

array([[ 153, 1101, 1186,    8],
       [   0, 1712,  445,  909],
       [  85, 1025,  495, 1208],
       [  45,  867,  191, 1631],
       [1302,   63, -146, 1425],
       [1139,  137, 1054,    0]])

In [50]:

act = rollout_actuation[5].tolist()
robot.pmove_xc(np.array(act*4))

In [19]:
# robot.pmove_xc(np.array([0,0,0,0]*4))

# Jar

In [94]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [95]:
# robot.execute_arm(Place_up_joint)
# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
# robot.execute_arm(init_joint)

In [114]:
Custom_grasp = np.load("actuations/Custom_grasp.npy")
Custom_grasp[0] = Custom_grasp[0]* 1.2
Custom_grasp[1:] = Custom_grasp[1:]* 1.6


Custom_grasp = Custom_grasp.tolist()
Custom_grasp

[[1366, 164, 1264, 0],
 [2083, 100, -233, 2280],
 [72, 1387, 305, 2609],
 [136, 1640, 792, 1932],
 [0, 2739, 712, 1454],
 [244, 1761, 1897, 12]]

In [115]:
robot.pmove_xc(np.tile(Custom_grasp[0],4))

In [113]:
for actuation in Custom_grasp:
    time.sleep(1)
    robot.pmove_xc(np.array(np.tile(actuation,4)))

for actuation in Custom_grasp:
    time.sleep(1)
    if actuation == Custom_grasp[-1]: break
    robot.pmove_xc(np.array(np.tile(actuation,4)))
    

In [63]:
actuation == Custom_grasp[-1]

True

In [48]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-51.04,-79.42,73.24,-83.68,-89.61,81.53])/90*DEG90
Target_down_joint = np.array([-50.75,-75,100.9,-115.94,-89.4,82.05])/90*DEG90
Place_up_joint = np.array([-38.01,-54.74,37.94,-73.06,-89.60,94.45])/90*DEG90
Place_down_joint = np.array([-38.07,-49.32,77.91,-118.45,-89.65,94.72])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


# robot.execute_arm(init_joint)

# robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

# robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# # 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# robot.execute_arm(Target_up_joint)
# robot.execute_arm(Place_up_joint)
# robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
# robot.execute_arm(Place_up_joint)
# robot.execute_arm(init_joint)

# Tumbler

In [43]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [44]:
# robot.execute_arm(Place_up_joint)
# init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
# robot.execute_arm(init_joint)

In [45]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-34.38,-90.85,93.59,-92.60,-89.61,98.27])/90*DEG90
Target_down_joint = np.array([-34.42,-87.67,107.73,-109.93,-89.65,98.36])/90*DEG90
Place_up_joint = np.array([-25.41,-66.39,63.93,-87.40,-89.59,107.17])/90*DEG90
Place_down_joint = np.array([-25.45,-56.45,89.79,-123.20,-89.68,107.37])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# # 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


# Tumbler2

In [46]:
robot.pmove_xc(np.array([0,0,0,0]*4))

In [47]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint = np.array([-11.76,-94.52,93.83,-89.17,-89.60,120.91])/90*DEG90
Target_down_joint = np.array([-11.76,-91.46,111.03,-109.48,-89.64,121.01])/90*DEG90
Place_up_joint = np.array([-8.21,-66.55,61.03,-84.34,-89.52,124.35])/90*DEG90
Place_down_joint = np.array([-8.26,-57.02,90.09,-122.93,-89.67,124.56])/90*DEG90

assert (Target_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([-1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([-1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))

# 들고
robot.pmove_xc(np.array([0,2000,3000,1000]*4))

# UnGrasp
robot.pmove_xc(np.array([1000,0,1000,0]*4))

# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1200,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,500,500]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))


robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


## Gat

In [48]:
robot.pmove_xc(np.array([0,0,0,0]*4))


In [49]:
init_joint=[0,-DEG90,DEG90,-DEG90,-DEG90,DEG90]
Target_up_joint   = np.array([16.84,-83.79,82.93,-89.01,-89.59,149.49])/90*DEG90
Target_down_joint = np.array([16.81,-82.13,96.38,-104.12,-89.61,149.57])/90*DEG90
Place_up_joint    = np.array([12.25,-55.41,42.79,-77.25,-89.58,144.78])/90*DEG90
Place_down_joint  = np.array([12.20,-48.98,77.21,-118.10,-89.63,145.01])/90*DEG90

assert (Target_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Target_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Place_up_joint * np.array([1,-1,1,-1,-1,1])>0).all()
assert (Place_down_joint * np.array([1,-1,1,-1,-1,1])>0).all()


robot.execute_arm(init_joint)

robot.execute_arm(Target_up_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))

robot.execute_arm(Target_down_joint)


# 접근 동작
robot.pmove_xc(np.array([2000,0,0,2000]*4))

# 잡는 동작
robot.pmove_xc(np.array([200,1500,0,2000]*4))

# 절반 돌리고
robot.pmove_xc(np.array([200,1200,1200,1600]*4))

# 나머지 돌리고
robot.pmove_xc(np.array([200,2200,500,1000]*4))


robot.execute_arm(Target_up_joint)
robot.execute_arm(Place_up_joint)
robot.execute_arm(Place_down_joint)

robot.pmove_xc(np.array([1000,0,1000,0]*4))
robot.execute_arm(Place_up_joint)
robot.execute_arm(init_joint)


Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan
Waiting for server...
Connected to server
Finish plan


In [50]:
robot.pmove_xc(np.array([0,0,0,0]*4))

# MAYO

In [17]:
pos = initpos+ PROPORTION_MATRIX @ np.array([1,0,1,0, 0,0,0,0, 0,0,0,0, 1,0,1,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0.3,0,1,0, 1,0,0,0.3, 0,0,0,0]) * 4000 * np.array([1,-1,-1,1]*4)

# pos = pos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,400, 0,0,0,0]) * np.array([1,-1,-1,1]*4)
snapbot.set_goalposcluster(pos,1)

In [18]:
pos = initpos+ PROPORTION_MATRIX @ np.array([1,0,1,0, 0,0,0,0, 0,0,0,0, 1,0,1,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0.3,0,1,0, 1,0,0,0.3, 0,0,0,0]) * 4000 * np.array([1,-1,-1,1]*4)
pos = pos    + PROPORTION_MATRIX @ np.array([0,0,0,0, 0,0,1,0, 1,0,0,0, 0,0,0,0]) * 1000 * np.array([1,-1,-1,1]*4)

# pos = pos + np.array([0,0,0,0, 0,0,0,0, 0,0,0,400, 0,0,0,0]) * np.array([1,-1,-1,1]*4)
snapbot.set_goalposcluster(pos,1)